# Задача 1

Найдем точечную оценку с помощью второго момента методом моментов
\begin{align*}
    \mathbb{E}_{\lambda}\left(X_1^2\right) = \overline{X^2} &\Leftrightarrow \\
    (16\lambda^{*})^2 + 16\lambda^{*} - \overline{X^2} = 0 &\Leftrightarrow \\
    \lambda^{*} = \frac{\sqrt{4 * \overline{X^2} + 1} - 1}{32}.
\end{align*}
Осталось лишь использовать ее для оценки истинного параметра $\lambda$.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import chi2

chat_id = 846523402 # Ваш chat ID, не меняйте название переменной

In [2]:
def solution(x: np.array) -> float:
    # Измените код этой функции
    # Это будет вашим решением
    # Не меняйте название функции и её аргументы
    return (np.sqrt(4 * (x ** 2).mean() + 1) - 1) / (2 * 16) # Ваш ответ

def experiment(N=1000, n=10, mu=10) -> float:
    from scipy.stats import poisson
    import numpy as np

    total = []
    for i in range(N):
        total.append((mu - solution(poisson.rvs(mu=16 * mu, size=n))) ** 2)
    total = np.array(total).mean()
    return total

In [3]:
experiment()

0.0661373385193838

# Задача 2

Итак, на вход нам мы ожидаем получить реализацию случайных величин:
\begin{align*}
    Z_i = \sqrt{X_i^2 + Y_i^2}, i=1,...,n,
\end{align*}
где $n$ - количество выпущенных Иваном стрел. Тогда заметим, что
\begin{align*}
    \sum\limits_{k=1}^{n} \frac{Z_k^2}{44\sigma^2} &= \sum\limits_{k=1}^{n} \left(\frac{X_i^2 + Y_i^2}{44\sigma^2}\right) \\
     &= \sum\limits_{k=1}^{n} \left(\left(\frac{X_i}{\sqrt{44\sigma^2}}\right)^2 + \left(\frac{Y_i}{\sqrt{44\sigma^2}}\right)^2\right) = \eta \sim {\cal{X}}_{2n}^2,
\end{align*}
т.е получили случайную величину, которая имеет распределение хи-квадрат с $2n$ степенями свободы (это распределение известное и уже не зависит от параметра!) Поэтому можно построить **точный** доверительный интервал уровня $1 - \alpha$ для неизвестного стандартного отклонения $\sigma$. Так как
\begin{align*}
    \mathbb{P}\left(x_{\alpha / 2, 2n}^2 < \eta < x_{1 - \alpha / 2, 2n}^2\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(x_{\alpha / 2, 2n}^2 < n \cdot \frac{\overline{Z^2}}{44\sigma^2} < x_{1 - \alpha / 2, 2n}^2\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2} < \sigma^2 < \frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}\right) = 1 - \alpha &\Leftrightarrow \\
    \mathbb{P}\left(\sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2}} < \sigma < \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}}\right)  = 1 - \alpha,
\end{align*}
где $x_{1 - \alpha / 2, 2n}^2$ и $x_{\alpha / 2, 2n}^2$ квантили распределения хи-квадрат с $2n$ степенями свободы уровней $1 - \alpha / 2$ и $\alpha / 2$ соответственно, а также
\begin{align*}
    \overline{Z^2} = \frac{1}{n} \cdot \sum\limits_{k=1}^{n} Z_k^2.
\end{align*}
Поэтому доверительный интервал для $\sigma$ при заданном уровне значимости $1 - \alpha$ имеет вид:
\begin{align*}
    \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{1 - \alpha / 2, 2n}^2}}, \sqrt{\frac{n\overline{Z^2}}{44 \cdot x_{\alpha / 2, 2n}^2}}
\end{align*}

In [4]:
def solution(p: float, x: np.array) -> tuple:
    alpha = 1 - p
    size = len(x)
    left = np.sqrt(size * (x ** 2).mean() / (44 * chi2.ppf(q=1 - alpha / 2, df=2 * size)))
    right = np.sqrt(size * (x ** 2).mean() / (44 * chi2.ppf(q=alpha / 2, df=2 * size)))
    
    return left, \
           right


def experiment(N: int, std: float) -> pd.DataFrame:
    std = std
    N = N

    alphas = [0.99, 0.9, 0.7, 0.9, 0.95, 0.9]
    sample_sizes = [1000, 1000, 100, 100, 10, 10]
    df = []

    data = zip(alphas, sample_sizes)

    for p, size in data:
        error_counter = 0
        ci_length = []
        
        for i in range(N):
            sample = np.sqrt(norm.rvs(loc=0, scale=np.sqrt(44 * std ** 2), size=size) ** 2 \
                + norm.rvs(loc=0, scale=np.sqrt(44 * std ** 2), size=size) ** 2)
            
            left, right = solution(p, sample)
            error_counter += not [left < std < right][0]
            ci_length.append(right - left)

        error_counter /= N
        ci_length = np.array(ci_length).mean()

        df.append([size, p, round(error_counter, 3), round(ci_length, 3)])
        
    df = pd.DataFrame(df, columns=['Размер выборки', 'Доверие', 'Частота ошибок', 'Длина интервала'])

    return df

In [5]:
experiment(5000, 1)

,Размер выборки,Доверие,Частота ошибок,Длина интервала
0,1000,0.99,0.010,0.082
1,1000,0.90,0.100,0.052
2,100,0.70,0.301,0.104
3,100,0.90,0.101,0.166
4,10,0.95,0.049,0.670
5,10,0.90,0.098,0.552


# Задача 1

In [67]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest


chat_id = 846523402 # Ваш chat ID, не меняйте название переменной

def solution(x_success: int, 
             x_cnt: int, 
             y_success: int, 
             y_cnt: int) -> bool:
    
    _, pval = proportions_ztest([x_success, y_success], [x_cnt, y_cnt], alternative='larger')
    effect = (pval < 0.1)
    
    return effect

In [69]:
data = [(1000, 0.14, 0.14), (3000, 0.14, 0.1050), (5000, 0.14, 0.1442)]

for n, p_1, p_2 in data:
    counter_effect = 0

    for _ in range(10000):
        control = sps.bernoulli.rvs(p=p_1, size=n) 
        test    = sps.bernoulli.rvs(p=p_2, size=n)

        control_success = sum(control)
        test_success    = sum(test)

        counter_effect += solution(control_success, n, test_success, n)

    if p_1 == p_2:
        print(counter_effect / n)
    else:
        print((n - counter_effect) / n)

0.996
-2.328666666666667
0.9402


# Задача 2

In [79]:
hystorical_data = pd.read_csv('historical_data.csv')
hystorical_data.head()

,Unnamed: 0,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x290,x291,x292,x293,x294,x295,x296,x297,x298,x299
0,0,0.484785,0.486611,0.422374,0.616654,0.525469,0.780493,0.725010,0.821293,0.497183,...,0.820166,0.445190,0.520859,0.339712,0.754967,0.193084,0.682776,0.797149,0.868285,0.760827
1,1,0.752387,0.667213,0.906403,0.383287,0.776146,0.751458,0.572882,0.898250,0.818333,...,0.743860,0.880095,0.174167,0.477711,0.327008,0.919090,0.753422,0.482342,0.870136,0.582859
2,2,0.570131,0.780968,0.803813,0.484459,0.823089,0.661533,0.299234,0.721643,0.116759,...,0.512036,0.598499,0.359185,0.937596,0.792427,0.419777,0.823381,0.474219,0.657657,0.315020
3,3,0.525048,0.624585,0.655140,0.397061,0.666091,0.781264,0.471400,0.815761,0.790411,...,0.688848,0.750622,0.769405,0.335168,0.420636,0.637075,0.813969,0.601138,0.647401,0.504314
4,4,0.801214,0.727864,0.848932,0.704623,0.619229,0.807593,0.857769,0.819997,0.580908,...,0.562761,0.574301,0.756689,0.597750,0.620371,0.776363,0.458411,0.726488,0.774714,0.849748
